In [1]:
import numpy as np
import pandas as pd
from math import cos, asin, sqrt

stops_time = pd.read_csv('Desktop/gtfs/stop_times.csv', sep=',')
stops=pd.read_csv('Desktop/gtfs/stops1.csv', sep=',')
trip=pd.read_csv('Desktop/gtfs/trips.csv', sep=',')

In [2]:
def distance(stop1,stop2):
    lat1,lon1=float(stops.loc[stops['stop_id']==stop1]['stop_lat']),float(stops.loc[stops['stop_id']==stop1]['stop_lon'])
    lat2,lon2=float(stops.loc[stops['stop_id']==stop2]['stop_lat']),float(stops.loc[stops['stop_id']==stop2]['stop_lon'])
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))*0.621371 #2*R*asin then to mile...

In [3]:
def updatedis(regionalid,dispre,disnext,terminal,dfdis):
    
    if regionalid not in list(dfdis['regionalid']):
        row=pd.DataFrame([[regionalid, dispre,disnext,terminal]],columns=['regionalid', 'dispre','disnext','terminal'])
        dfdis=dfdis.append(row)
    else:
        if dispre > float(dfdis.loc[dfdis['regionalid']==regionalid][['dispre']]['dispre']):
            dfdis.loc[dfdis['regionalid']==regionalid,'dispre']=dispre
        if disnext > float(dfdis.loc[dfdis['regionalid']==regionalid][['disnext']]['disnext']):
            dfdis.loc[dfdis['regionalid']==regionalid,'disnext']=disnext         
    return dfdis 

In [4]:
rou_trip={1:[1]}
for i in range(trip.shape[0]):
    i=i+1
    if trip.route_id[i]==trip.route_id[i-1]:
        rou_trip[int(trip.route_id[i])].append(int(trip.trip_id[i]))
    else:
        rou_trip[int(trip.route_id[i])]=[int(trip.trip_id[i])]

ValueError: invalid literal for int() with base 10: 'BLUE'

In [5]:
stops_time.groupby(['trip_id']).count()
grouped=stops_time.groupby(['trip_id'])
summed=grouped.count()
summed.head()

,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
trip_id,,,,,,,
1,55,55,55,55,55,55,55
2,55,55,55,55,55,55,55
3,55,55,55,55,55,55,55
4,55,55,55,55,55,55,55
5,55,55,55,55,55,55,55


In [6]:
tripid=[]
for route in rou_trip:
    standard=[summed.stop_id[rou_trip[route][0]]]
    tripid.append(rou_trip[route][0])
    print(route,':',rou_trip[route][0],':',standard)
    for tripn in rou_trip[route][1:]:
        if summed.stop_id[tripn] not in standard:
            standard.append(summed.stop_id[tripn])
            tripnew=tripn
            print(route,':',tripnew,':',summed.stop_id[tripn])
            tripid.append(tripnew)

1 : 1 : [55]
1 : 224 : 53
2 : 438 : [69]
2 : 653 : 68
3 : 869 : [43]
3 : 874 : 45
4 : 949 : [9]
4 : 963 : 7
5 : 977 : [21]
5 : 981 : 52
5 : 1010 : 54
6 : 1033 : [40]
6 : 1081 : 37
6 : 1093 : 42
7 : 1133 : [58]
7 : 1273 : 60
8 : 1405 : [31]
8 : 1447 : 66
8 : 1529 : 40
8 : 1551 : 69
9 : 1667 : [34]
9 : 1668 : 44
9 : 1674 : 79
9 : 1678 : 11
9 : 1686 : 69
9 : 1703 : 41
9 : 1705 : 50
9 : 1716 : 10
9 : 1726 : 70
10 : 1748 : [19]
10 : 1822 : 27
10 : 2155 : 32
11 : 2559 : [31]
12 : 2683 : [33]
12 : 2816 : 41
13 : 2963 : [19]
13 : 2964 : 27
13 : 2970 : 34
14 : 2977 : [55]
14 : 3001 : 50
15 : 3025 : [36]
15 : 3035 : 43
16 : 3045 : [9]
16 : 3055 : 28
16 : 3129 : 8
16 : 3137 : 30
17 : 3209 : [18]
17 : 3212 : 22
17 : 3313 : 20
18 : 3409 : [80]
18 : 3417 : 62
19 : 3425 : [41]
19 : 3449 : 45
20 : 3489 : [48]
21 : 3557 : [42]
21 : 3577 : 40
22 : 3609 : [39]
22 : 3629 : 40
23 : 3653 : [47]
24 : 3697 : [39]
24 : 3721 : 37
25 : 3745 : [60]
25 : 3765 : 57
26 : 3793 : [38]
26 : 3805 : 37
27 : 3817 : [50]
2

In [7]:
nolist=[]
dfdis=pd.DataFrame(columns=['regionalid', 'dispre','disnext','terminal'])
for maxtrip in tripid:
    try438=stops_time.loc[stops_time['trip_id']==maxtrip][['stop_id','stop_sequence']]
    print(maxtrip)
    for j in range(try438.shape[0]-1):
        
        if stops.loc[stops['stop_id']==try438.iloc[j][0]].empty or stops.loc[stops['stop_id']==try438.iloc[j+1][0]].empty:
            nolist.append(try438.iloc[j][0])
            continue
        
        regionalid=int(stops.loc[stops['stop_id']==try438.iloc[j][0]][['stop_code']]['stop_code'])
        
        indexfirst=try438.iloc[j][0]
        indexsecond=try438.iloc[j+1][0]
        dist=distance(indexfirst,indexsecond)
        
        if j ==0:
            disprevious=0
            disnext=dist
            dist_qianyige=dist
            terminal="Y"
        else:
            disprevious=dist_qianyige
            disnext=dist
            dist_qianyige=dist
            terminal="N"
        dfdis=updatedis(regionalid,disprevious,disnext,terminal,dfdis)
        #dfdis=df2
        print (j)
    if stops.loc[stops['stop_id']==try438.iloc[-1][0]].empty:
        nolist.append(try438.iloc[-1][0])
        continue
    regionalid=int(stops.loc[stops['stop_id']==try438.iloc[-1][0]][['stop_code']] ['stop_code'])   
    disnext=0
    disprevious=dist_qianyige
    terminal="Y"
    dfdis=updatedis(regionalid,disprevious,disnext,terminal,dfdis)

1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
224
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
438
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
653
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
869
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
874
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
949
0
1
2
3
4
5
6
7
963
0
1
2
3
4
5
977
0
1
2
3
4
5
6
7
8

18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
4520
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
4570
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
4586
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
4659
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
4661
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
4795
0
1
2
3
4
5
6
7
8
9
10
11
4843
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
4883
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
4903
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
4923

39
40
41
42
43
44
45
46
10397
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
10637
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
10866
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
11082
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
11206
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
11322
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
11514
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
11698
0
1
2
3
4
5
6
7
8
9
10
11
12
11742
0
1
2
3
4
5
6
7
8
9
10
11
12
13
11786
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
11787
0
1
2
3
4
5

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
23504
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
23712
0
1
2
3
4
5
6
7
8
9
10
11
12
23719
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
23720
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
23922
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23938
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
23939
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61

28
29
30
31
32
33
34
35
36
37
31530
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
31549
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
32019
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
32020
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
32026
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
32034
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
32472
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
32504
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
32548
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
32596
0
1
2
3
4
5
6
7
8
9
10
11

15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
35605
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
35612
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
35669
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
35670
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
35765
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
44749
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
44820
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
44823
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
44947
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
44948
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
51068
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
51371
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
51373
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
51753
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
51755
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
51813
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
52141
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52201
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
61048
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
61081
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
61106
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
61277
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
61347
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
61441
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
61477
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23

In [9]:
dfdis.to_csv("Desktop/dfdis2.csv")

In [10]:
m11 = pd.read_csv('H:/HRao/data/m11.csv', sep=',')
#m11.head()
dismatch=pd.read_csv("Desktop/dfdis2.csv",sep=',')
stopindc=list(m11['REG_ID'])

In [11]:
m11=m11.rename(columns = {'diststart':'dispre'})
m11=m11.rename(columns = {'disprevious':'disnext'})
m11.head()

,Unnamed: 0,REG_ID,LATITUDE,LONGITUDE,ON_STR,AT_STR,X.1,WARD,ANC_ID,SMD,...,snow.removal.Company,Circulator.LINE,Circulator.ID,tripperstopday,dispre,disnext,ridership.y,sum,active,terminal
0,0,1000001,38.816082,-77.017741,SHEPHERD PKY SW,SHEPHERD PKY SW,NaN,8,8D,SMD 8D04,...,,NaN,NaN,125.0,NaN,NaN,1.35,0.0,Y,NaN
1,1,1000002,38.816204,-77.016128,DC VILLAGE SE,PARKING LOT SE,NaN,8,8D,SMD 8D04,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2,1000003,38.816294,-77.018109,SHEPHERD PKY SW,DC VILLAGE LN,NaN,8,8D,SMD 8D04,...,,NaN,NaN,64.0,NaN,NaN,0.64,0.0,Y,NaN
3,3,1000004,38.819635,-77.010821,MARTIN LUTHER KING JR AVE SW,JOLIET ST SW,NaN,8,8D,SMD 8D02,...,,NaN,NaN,160.0,NaN,NaN,82.25,0.0,Y,NaN
4,4,1000005,38.820020,-77.016605,SHEPHERD PKY SW,FIRE DEPARTMENT TRAINING CENTER,NaN,8,8D,SMD 8D04,...,,NaN,NaN,63.0,NaN,NaN,12.94,0.0,Y,NaN


In [12]:
for ID in dismatch['regionalid']:
    if ID in stopindc:
    #ID=dismatch['regionalid']
        m11.loc[m11['REG_ID']==ID,'dispre'] = float(dismatch.loc[dismatch['regionalid']==ID]['dispre'])
        m11.loc[m11['REG_ID']==ID,'disnext'] = float(dismatch.loc[dismatch['regionalid']==ID]['disnext'])
        m11.loc[m11['REG_ID']==ID,'terminal'] = str(dismatch.loc[dismatch['regionalid']==ID]['terminal'])

In [ ]:
m11.to_csv('H:/HRao/data/m11.csv')

In [14]:
m11Y=m11.loc[m11['active']=='Y']
m11nodis=m11Y.loc[m11Y['dispre'].isnull()]
#list(m11nodis.REG_ID)

In [15]:
stopid=[]
for regid in list(m11nodis.REG_ID):
    stopid.append(int(stops.loc[stops['stop_code']==regid]['stop_id']))

In [19]:

for sid in stopid:
    #list(stops_time.loc[stops_time['stop_id']==sid].index)
    A=stops_time.loc[stops_time['stop_id']==sid]
    B=set(A['stop_sequence'])
    for i in range(len(B)):
        seq=B.pop()
        C=A.loc[A['stop_sequence']==seq]
        tripid=list(A.loc[A['stop_sequence']==seq]['trip_id'])[0]
        k=list(C.loc[C['trip_id']==tripid].index)[0]#index
        stop_p=stops_time.iloc[k-1]['stop_id']
        stop_0=stops_time.iloc[k]['stop_id']
        stop_n=stops_time.iloc[k+1]['stop_id']
        if stops.loc[stops['stop_id']==stop_p].empty or stops.loc[stops['stop_id']==stop_n].empty:
            continue
        dispre = distance(stop_p,stop_0)
        disnext = distance(stop_0,stop_n)
        dispre_pre=dispre
        disnext_pre=disnext
        if dispre>dispre_pre:
            dispre_f=dispre
        else:
            dispre_f=dispre_pre
        if disnext>disnext_pre:
            disnext_F=disnext
        else:
            disnext_f=disnext_pre
        if seq==0:
            term="Y"
        else:
            term='N'
    regionalid=int(stops.loc[stops['stop_id']==sid][['stop_code']]['stop_code'])
#print(regionalid,dispre_f,disnext_f)
    m11.loc[m11['REG_ID']==regionalid,'dispre'] = dispre_f
    m11.loc[m11['REG_ID']==regionalid,'disnext'] = disnext_f
    m11.loc[m11['REG_ID']==regionalid,'terminal'] = term

In [20]:
m11.to_csv('H:/HRao/data/m13.csv')